In [ ]:
# @title Initial Setup
## Authenticate your Google account to access Google Cloud services. This will allow you to interact with Google Cloud Storage using the credentials associated with your account.
## Also import your Google Drive to access your team's Shared Google Drive as well as Operating System
!pip install rasterio
!pip install dbfread
from google.colab import auth
from google.colab import drive
from google.cloud import storage
import os
auth.authenticate_user()
drive.mount('/content/drive/', force_remount=True)


client = storage.Client(project='HDSI-Agri-Datathon-2024')
bucket = client.bucket('hdsi-agri-prompt-data')

Mounted at /content/drive/


In [ ]:
# @title Parsing data to distinguish crops
# This is how we determined what readings in CropScape are crops and what wasn't

import dbfread
import os
import pandas as pd

# We had actually downloaded parts of the database while experimenting with data
directory_path = "/content/drive/MyDrive/HDSI/HDSI2/1/"
folder_name = "cropland_images/WallaWalla/CDL_2006_53071.tif.vat.dbf"

dbf_file = os.path.join(directory_path, folder_name)

# Load the DBF file
table = dbfread.DBF(dbf_file)

df = pd.DataFrame(iter(table))

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Remove background data from file
df_cleaned = df[~((df['RED'] == 0.000) & (df['GREEN'] == 0.000) & (df['BLUE'] == 0.000))]

# Values of non-crop vegetation - these must be ignored
non_crop = [81, 82, 83, 87, 88, 92, 111, 112, 121, 122, 123, 124, 131, 141, 142, 143, 151, 176, 190, 195]

# Create a new column 'is_crop' with 1 if it's a crop, 0 if not a crop
df_cleaned['is_crop'] = df_cleaned['VALUE'].apply(lambda x: 0 if x in non_crop else 1)

print(df_cleaned)

     VALUE    RED  GREEN   BLUE                   CLASS_NAME  OPACITY  is_crop
1        1  1.000  0.827  0.000                         Corn      1.0        1
2        2  1.000  0.149  0.149                       Cotton      1.0        1
3        3  0.000  0.659  0.894                         Rice      1.0        1
4        4  1.000  0.620  0.043                      Sorghum      1.0        1
5        5  0.149  0.439  0.000                     Soybeans      1.0        1
6        6  1.000  1.000  0.000                    Sunflower      1.0        1
10      10  0.439  0.647  0.000                      Peanuts      1.0        1
11      11  0.000  0.686  0.294                      Tobacco      1.0        1
12      12  0.867  0.647  0.043                   Sweet Corn      1.0        1
13      13  0.867  0.647  0.043              Pop or Orn Corn      1.0        1
14      14  0.494  0.827  1.000                         Mint      1.0        1
21      21  0.886  0.000  0.486                     

/tmp/ipykernel_86276/3889545136.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['is_crop'] = df_cleaned['VALUE'].apply(lambda x: 0 if x in non_crop else 1)


In [ ]:
# @title Printing out spatial resolution of a Cropscape image
# This code was used when we were experimenting with discovering the spatial resolution of the tif images
import rasterio

with rasterio.open('/content/drive/MyDrive/HDSI/HDSI2/1/cropland_images/WallaWalla/CDL_2021_53071.tif') as img:
    # This gets the spatial resolution of each pixel
    resolution = img.res

    # Gets the total image size of the full image
    width = img.width
    height = img.height


print("CROPSCAPE")
print(f"Each pixel represents: {resolution[0] * resolution[1]}m^2")
print(f"Resolution: {resolution[0]} x {resolution[1]} (pixel width x pixel height)")
print(f"Width: {width} Height: {height}")
print(f"Meters Width: {width*resolution[0]} Height: {height*resolution[1]}")

CROPSCAPE
Each pixel represents: 900.0m^2
Resolution: 30.0 x 30.0 (pixel width x pixel height)
Width: 2681 Height: 2431
Meters Width: 80430.0 Height: 72930.0


In [ ]:
# @title Printing out spatial resolution of a VegScape image
# This code was used when we were experimenting with discovering the spatial resolution of the tif images

import rasterio
import numpy


with rasterio.open('/content/drive/MyDrive/HDSI/HDSI2/1/NDVI_images/Adams_2001_01_NDVI.tif') as img:
    # This gets the spatial resolution of each pixel
    resolution = img.res
    # Gets the total image size of the full image
    width = img.width
    height = img.height


print("VEGSCAPE")
print(f"Each pixel represents: {resolution[0] * resolution[1]}m^2")
print(f"Resolution: {resolution[0]} x {resolution[1]} (pixel width x pixel height)")
print(f"Pixels Width: {width} Height: {height}")
print(f"Meters Width: {width*resolution[0]} Height: {height*resolution[1]}")

VEGSCAPE
Each pixel represents: 62500.0m^2
Resolution: 250.0 x 250.0 (pixel width x pixel height)
Pixels Width: 474 Height: 281
Meters Width: 118500.0 Height: 70250.0


In [ ]:
# @title Code used for rescaling VegScape images

import rasterio
from rasterio.enums import Resampling

def rescaleVegScape(vegScapeFilePath, cropScapeFilePath):
    vegScapeFile = f'gs://hdsi-agri-prompt-data/{vegScapeFilePath}'
    cropScapeFile = f'gs://hdsi-agri-prompt-data/{cropScapeFilePath}'
    # Open the VegScape and CropScape image
    with rasterio.open(vegScapeFile) as vegscape:
        with rasterio.open(cropScapeFile) as cropscape:

            # Resize the VegScape image to match CropScape dimensions
            new_width = cropscape.width
            new_height = cropscape.height

            # Resample the data to the new dimensions
            data_resampled = vegscape.read(
                out_shape=(vegscape.count, new_height, new_width),
                resampling=Resampling.bilinear
            )

            # Return the rescaled data
            return data_resampled[0]


In [ ]:
# @title

import pandas as pd
import rasterio
from google.cloud import storage

# Array stores what week numbers belong to each month
# Used to determine what week numbers each month should have
# This does not account for granular details (leap year, what day of week year starts on, etc.)
# January gets first 4 weeks, then February gets next 4, etc.
weeks = [0, 4, 8, 12, 17, 21, 25, 30, 35, 39, 44, 48, 52]

def getNDVIForMonth(cropCoordinates, month, year, countyName, fipsCode):
    validFilePaths = []
    rescaledFiles = []

    cropScapeFilePath = f'HDSI_AGRI_Prompt_1/cropland_images/{countyName}/CDL_{year}_{fipsCode}.tif'

    weekRange = range(weeks[month-1] + 1, weeks[month] + 1)

    # Find all weeks that have scans within the month
    for i in weekRange:
      filePath = f'HDSI_AGRI_Prompt_1/NDVI_images/{countyName}_{year}_{str(i).zfill(2)}_NDVI.tif'
      vegScapeFile = bucket.blob(filePath)
      if (vegScapeFile.exists()):
        validFilePaths.append(filePath)

    # Rescale valid VegScape scans to match with CropScape coordinates
    for filePath in validFilePaths:
      rescaledFiles.append(rescaleVegScape(filePath, cropScapeFilePath))

    # Stores all NDVI averages from all weeks in month for reach crop pixel location
    NDVI_average = []

    n = len(rescaledFiles)

    # Iterrate through all crop locations
    for index, row in cropCoordinates.iterrows():
      x = row['x']
      y = row['y']
      average = 0
      for array in rescaledFiles:
        average += array[x, y]
      average //= n
      NDVI_average.append(average)

    cropCoordinates[str(month)] = NDVI_average



    return cropCoordinates  # Return the modified DataFrame


In [ ]:
import pandas as pd
import rasterio
from io import BytesIO

# Stores all non crop values discovered previously
non_crop = [0, 81, 82, 83, 87, 88, 92, 111, 112, 121, 122, 123, 124, 131, 141, 142, 143, 151, 176, 190, 195]

# Find crop locations for the whole year
def findCropLocations(cropScapeFilePath):

    cropScapeFile = f'gs://hdsi-agri-prompt-data/{cropScapeFilePath}'
    columns = ['x', 'y']

    # Create a list to collect pixel coordinates
    cropLocations_list = []

    with rasterio.open(cropScapeFile) as county:
        pixelData = county.read(1)  # Read the data in file
        # Iterate through each pixel
        for row in range(pixelData.shape[0]):
            for col in range(pixelData.shape[1]):
                pixel_value = pixelData[row, col]  # Get the pixel value

                # Check if the pixel value is in the non_crop list
                if pixel_value not in non_crop:
                    # Append the pixel coordinates (row, column)
                    cropLocations_list.append((row, col))

        # Create a DataFrame from the collected pixel coordinates
        cropLocations = pd.DataFrame(cropLocations_list, columns=columns)

    return cropLocations

In [ ]:
import numpy as np
import pandas as pd

countyName = 'Gem'
years = ['2007', '2010', '2011', '2012', '2016', '2017', '2018', '2021', '2022']
fips = '16045'
months = 12
numOfYears = len(years)

# Stores data of all years
data = np.empty((numOfYears, months + 1))

for yearIndex in range(len(years)):
    # Path for cropscape file for year
    croplandPath = f"HDSI_AGRI_Prompt_1/cropland_images/{countyName}/CDL_{years[yearIndex]}_{fips}.tif"

    cropCoords = findCropLocations(croplandPath)

    for i in range(1, 13):
        cropCoords = getNDVIForMonth(cropCoords, i, years[yearIndex], countyName, fips)

    runningAverages = np.zeros(12)

    for index, row in cropCoords.iterrows():
        for i in range(12):
            runningAverages[i] += row[str(i + 1)]

    for i in range(12):
        runningAverages[i] //= cropCoords.shape[0]

    totalAverage = 0

    for i in range(12):
        totalAverage += runningAverages[i]

    totalAverage //= 12

    for i in range(12):
        data[yearIndex][i] = runningAverages[i]

    data[yearIndex][12] = totalAverage

# Convert the NumPy array to a DataFrame for output
columns = [str(i) for i in range(1, 13)] + ['Total Average']
df = pd.DataFrame(data, index=years, columns=columns)

# Print the DataFrame
print(df)


          1      2      3      4      5      6      7      8      9     10  \
2007  161.0  158.0  163.0  187.0  191.0  183.0  175.0  170.0  167.0  172.0   
2010  141.0  147.0  169.0  176.0  195.0  191.0  190.0  177.0  169.0  166.0   
2011  136.0  149.0  151.0  178.0  186.0  189.0  187.0  174.0  172.0  173.0   
2012  162.0  158.0  157.0  183.0  191.0  189.0  178.0  175.0  168.0  163.0   
2016  147.0  163.0  176.0  201.0  199.0  189.0  176.0  173.0  169.0  166.0   
2017  125.0  143.0  172.0  197.0  205.0  195.0  182.0  175.0  173.0  174.0   
2018  160.0  162.0  160.0  186.0  199.0  195.0  179.0  171.0  167.0  163.0   
2021  148.0  140.0  158.0  178.0  189.0  189.0  181.0  180.0  178.0  173.0   
2022  131.0  145.0  169.0  188.0  203.0  203.0  190.0  177.0  175.0  167.0   

         11     12  Total Average  
2007  176.0  150.0          171.0  
2010  149.0  132.0          166.0  
2011  176.0  174.0          170.0  
2012  165.0  151.0          170.0  
2016  174.0  139.0          172.0  
201